# UploadBaseDados DynamoxSet24
Upload dass bases de dados Dynamox do GitHub.

## Codigos para iniciar o trabalho
A seguir estão os pacotes requeridos para tratamento dos dados

In [ ]:
%pip install markdown
%pip install statsmodels
%pip install scikit-learn
%pip install PyGithub
%pip install gitpython
%pip install statsmodels
%pip install dash
%pip install xlwt
%pip install openpyxl
%pip install tensorflow
%pip install scipy
%pip install arch


# Atualizar pacotes
import numpy as np
import os
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from IPython.display import Image
from sklearn.svm import SVR
import tensorflow as tf


# Baixar e Concatenar os arquivos em um unico DF

In [ ]:
import os
import pandas as pd
import requests
from io import BytesIO
from zipfile import ZipFile

# URL para o repositório GitHub (baseado em arquivos públicos)
repo_url = "https://github.com/CidClayQuirino/DataBaseDynamox/archive/refs/heads/main.zip"

# Baixar e extrair os arquivos do repositório
response = requests.get(repo_url)
zip_file = ZipFile(BytesIO(response.content))
zip_file.extractall("DataBaseDynamox")

# Caminho para os arquivos extraídos
folder_path = "DataBaseDynamox/DataBaseDynamox-main/"

# Inicializar uma lista para armazenar os dataframes
df_list = []

# Função para consolidar as abas Acceleration, Velocity e Temperature
def consolidar_abas(file_path, file_name):
    # Inicializar um dataframe vazio
    df_consolidado = pd.DataFrame()

    # Tentar abrir o arquivo e verificar as abas disponíveis
    try:
        xls = pd.ExcelFile(file_path)
        sheet_names = xls.sheet_names  # Obter o nome das abas
    except Exception as e:
        print(f"Erro ao abrir o arquivo '{file_name}': {e}")
        return df_consolidado  # Retorna um dataframe vazio em caso de erro

    # Verificar e consolidar os dados de cada aba
    for aba in ['Acceleration', 'Velocity', 'Temperature']:
        if aba in sheet_names:  # Verificar se a aba existe
            try:
                # Ler a aba específica
                df_aba = pd.read_excel(file_path, sheet_name=aba)

                # Normalizar nomes de colunas
                df_aba.rename(columns=lambda x: x.strip().lower(), inplace=True)

                # Verificar se a coluna 'datetime' existe
                if 'datetime' not in df_aba.columns:
                    raise ValueError(f"'datetime' column not found in sheet {aba} in file {file_name}")

                # Verificar colunas específicas de acordo com cada aba
                if aba.lower() == 'acceleration':
                    colunas_acceleration = ['accelerationx', 'accelerationy', 'accelerationz']
                    for col in colunas_acceleration:
                        if col not in df_aba.columns:
                            df_aba[col] = None  # Adicionar a coluna se não existir no arquivo
                elif aba.lower() == 'velocity':
                    colunas_velocity = ['velocityx', 'velocityy', 'velocityz']
                    for col in colunas_velocity:
                        if col not in df_aba.columns:
                            df_aba[col] = None  # Adicionar a coluna se não existir no arquivo
                elif aba.lower() == 'temperature' and 'temperature' not in df_aba.columns:
                    df_aba['temperature'] = None  # Adicionar a coluna se não existir

                # Selecionar as colunas de interesse
                df_aba = df_aba[['datetime'] + [col for col in df_aba.columns if col in ['accelerationx', 'accelerationy', 'accelerationz', 'velocityx', 'velocityy', 'velocityz', 'temperature']]]

                # Mesclar com o dataframe consolidado
                if df_consolidado.empty:
                    df_consolidado = df_aba
                else:
                    df_consolidado = pd.merge(df_consolidado, df_aba, on='datetime', how='outer')

            except Exception as e:
                print(f"Erro ao processar a aba '{aba}' no arquivo '{file_name}': {e}")
        else:
            continue

    # Adicionar uma coluna com o nome do arquivo e remover '.xlsx' para ser o nome do componente
    df_consolidado['Componente'] = file_name.replace('.xlsx', '')

    return df_consolidado

# Percorrer os arquivos na pasta extraída
for file_name in os.listdir(folder_path):
    if file_name.endswith('.xlsx'):
        # Caminho completo para o arquivo
        file_path = os.path.join(folder_path, file_name)

        # Consolidar as abas de interesse
        df_consolidado = consolidar_abas(file_path, file_name)

        # Adicionar o dataframe consolidado à lista
        df_list.append(df_consolidado)

# Concatenar todos os dataframes em um único dataframe
DfDynamox = pd.concat(df_list, ignore_index=True)

# Renomear a coluna 'FileName' para 'Componente' (se necessário, caso não tenha sido feita na função)
DfDynamox.rename(columns={'FileName': 'Componente'}, inplace=True)

In [ ]:
import pandas as pd

# Identificar todas as colunas que contêm 'datetime' no nome
datetime_columns = [col for col in DfDynamox.columns if 'datetime' in col]
# Aplicar a conversão para cada coluna que contenha 'datetime'
for col in datetime_columns:
    DfDynamox[col] = pd.to_datetime(DfDynamox[col]).dt.strftime('%Y-%m-%d %H:%M:%S')
# Exibir o DataFrame ajustado
DfDynamox.head()

,datetime,accelerationx,accelerationy,accelerationz,velocityx,velocityy,velocityz,temperature,Componente
0,2024-08-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,EM3401_SwingGear_Front
1,2024-08-18 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,EM3401_SwingGear_Front
2,2024-09-08 16:27:27,0.040428,0.071845,0.054609,3.662771,8.057280,4.818910,43.831158,EM3401_SwingGear_Front
3,2024-09-08 18:07:27,0.208048,0.292167,0.214138,24.220568,9.819565,25.127970,43.435951,EM3401_SwingGear_Front
4,2024-09-08 20:07:27,0.061934,0.131947,0.060289,8.206766,3.184391,4.031892,42.805752,EM3401_SwingGear_Front


In [ ]:
import plotly.graph_objects as go
import pandas as pd

# Filtrar linhas onde o nome do componente contenha 'EM3401'
DfDynamox_EM3401 = DfDynamox[DfDynamox['Componente'].str.contains('EM3401', case=False)]

# Definir os componentes a serem comparados
componentes = [
    'EM3401_BoomCylinder_LD',
    'EM3401_BoomCylinder_LE',
    'EM3401_FinalDrive_LD',
    'EM3401_FinalDrive_LE',
    'EM3401_MainPumpP1',
    'EM3401_MainPumpP2',
    'EM3401_SwingDrive_Rear',
    'EM3401_SwingGear_Front',
    'EM3401_SwingGear_Rear',
    'EM3401_SwingMotor_Front',
    'EM3401_SwingMotor_Rear'
]

# Definir os parâmetros disponíveis para visualização
parametros = ['accelerationx', 'accelerationy', 'accelerationz',
              'velocityx', 'velocityy', 'velocityz', 'temperature']

# Função para gerar gráficos com botões horizontais e exibir nomes dos componentes nas legendas
def gerar_graficos_com_botoes(DfDynamox_filtered):
    fig = go.Figure()

    # Obter a data mínima e máxima dos dados filtrados
    data_inicio = pd.to_datetime("2024-09-01")  # Forçar data inicial para 1º de setembro
    data_fim = DfDynamox_filtered['datetime'].max()

    # Iterar pelos componentes e parâmetros e adicionar traces ocultos inicialmente
    for componente in componentes:
        for parametro in parametros:
            fig.add_trace(
                go.Scatter(x=DfDynamox_filtered[DfDynamox_filtered['Componente'] == componente]['datetime'],
                           y=DfDynamox_filtered[DfDynamox_filtered['Componente'] == componente][parametro],
                           mode='lines', name=f'{componente} - {parametro}', visible=False)
            )

    # Tornar o primeiro gráfico visível por padrão
    for trace in fig.data:
        if 'accelerationx' in trace.name:
            trace.visible = True

    # Criar um botão para cada parâmetro
    buttons = []
    for parametro in parametros:
        buttons.append(
            dict(
                label=parametro,
                method='update',
                args=[{'visible': [parametro in trace.name for trace in fig.data]},
                      {'title': f'Comparação de {parametro.capitalize()} entre os Componentes'}]
            )
        )

    # Configurar layout com menu de botões na posição direita, acima da legenda
    fig.update_layout(
        updatemenus=[dict(
            type="buttons",
            direction="down",  # Botões na vertical
            buttons=buttons,
            pad={"r": 10, "t": 10},  # Espaçamento
            x=1.15,  # Posicionar à direita
            y=0.5,  # Alinhar verticalmente ao topo do gráfico
            xanchor='left',
            yanchor='top',
            font=dict(size=12),
            bgcolor='lightgray',  # Cor de fundo do menu
            bordercolor='black',
            borderwidth=2
        )],
        height=800,
        title="Comparação de Parâmetros entre os Componentes EM3401",
        title_font=dict(size=24),
        showlegend=True,  # Exibir legenda com os nomes dos componentes
        legend_title_text="Componentes",  # Título para a legenda
        xaxis_title="Datetime",
        xaxis=dict(
            tickfont=dict(size=14),
            range=[data_inicio, data_fim],  # Definir o range com base na data inicial ajustada e na data máxima
            constrain="domain"  # Garante que o range do eixo não ultrapasse as datas mín/máx
        ),
        yaxis=dict(title="Valor", titlefont=dict(size=16), tickfont=dict(size=14)),
        template='plotly_white',
        font=dict(size=12)
    )

    # Exibir o gráfico
    fig.show()

# Filtrar os dados para os componentes no DfDynamox_EM3401
DfDynamox_filtered = DfDynamox_EM3401[DfDynamox_EM3401['Componente'].isin(componentes)]

# Gerar gráficos com botões horizontais e exibir os nomes dos componentes nas linhas
gerar_graficos_com_botoes(DfDynamox_filtered)



In [ ]:
import plotly.graph_objects as go
import pandas as pd

# Filtrar linhas onde o nome do componente contenha '6030'
DfDynamox_6030 = DfDynamox[DfDynamox['Componente'].str.contains('6P3', case=False)]

# Definir os componentes da 6030 a serem comparados
componentes = [
    '6P3000101_Redutor Giro LD',
    '6P3000101_Redutor Giro LE',
    '6P3000111_Redutor Giro LD',
    '6P3000111_Redutor Giro LE'
]

# Definir os parâmetros disponíveis para visualização
parametros = ['accelerationx', 'accelerationy', 'accelerationz',
              'velocityx', 'velocityy', 'velocityz', 'temperature']

# Função para renomear os componentes de acordo com o padrão desejado
def renomear_componente(componente):
    if '6P3000101' in componente:
        return componente.replace('6P3000101', '6030-01')
    elif '6P3000111' in componente:
        return componente.replace('6P3000111', '6030-02')
    return componente

# Função para gerar gráficos com botões horizontais, posicionados à direita acima da legenda
def gerar_graficos_com_botoes(DfDynamox_filtered):
    fig = go.Figure()

    # Obter a data mínima e máxima dos dados filtrados
    data_inicio = pd.to_datetime("2024-09-12")  # Forçar a data inicial para 1º de setembro
    data_fim = DfDynamox_filtered['datetime'].max()

    # Iterar pelos componentes e parâmetros e adicionar traces ocultos inicialmente
    for componente in componentes:
        for parametro in parametros:
            componente_renomeado = renomear_componente(componente)
            fig.add_trace(
                go.Scatter(x=DfDynamox_filtered[DfDynamox_filtered['Componente'] == componente]['datetime'],
                           y=DfDynamox_filtered[DfDynamox_filtered['Componente'] == componente][parametro],
                           mode='lines', name=f'{componente_renomeado} - {parametro}', visible=False)
            )

    # Tornar o primeiro gráfico visível por padrão
    for trace in fig.data:
        if 'accelerationx' in trace.name:
            trace.visible = True

    # Criar um botão para cada parâmetro
    buttons = []
    for parametro in parametros:
        buttons.append(
            dict(
                label=parametro,
                method='update',
                args=[{'visible': [parametro in trace.name for trace in fig.data]},
                      {'title': f'Comparação de {parametro.capitalize()} entre os Componentes'}]
            )
        )

    # Configurar layout com menu de botões na posição direita, acima da legenda
    fig.update_layout(
        updatemenus=[dict(
            type="buttons",
            direction="down",  # Botões na vertical
            buttons=buttons,
            pad={"r": 10, "t": 10},  # Espaçamento
            x=1.15,  # Posicionar à direita
            y=0.5,  # Alinhar verticalmente ao topo do gráfico
            xanchor='left',
            yanchor='top',
            font=dict(size=12),
            bgcolor='lightgray',  # Cor de fundo do menu
            bordercolor='black',
            borderwidth=2
        )],
        height=800,
        title="Comparação de Parâmetros entre os Componentes 6030",
        title_font=dict(size=24),
        showlegend=True,  # Exibir legenda com os nomes dos componentes
        legend_title_text="Componentes",  # Título para a legenda
        xaxis_title="Datetime",
        xaxis=dict(
            tickfont=dict(size=14),
            range=[data_inicio, data_fim],  # Definir o range com base na data inicial ajustada e na data máxima
            constrain="domain"  # Garante que o range do eixo não ultrapasse as datas mín/máx
        ),
        yaxis=dict(title="Valor", titlefont=dict(size=16), tickfont=dict(size=14), range=[0, None]),  # Definir mínimo do eixo Y como 0
        template='plotly_white',
        font=dict(size=12)
    )

    # Exibir o gráfico
    fig.show()

# Filtrar os dados para os componentes no DfDynamox_6030
DfDynamox_filtered = DfDynamox_6030[DfDynamox_6030['Componente'].isin(componentes)]

# Gerar gráficos com botões horizontais à direita, acima da legenda
gerar_graficos_com_botoes(DfDynamox_filtered)
